In [6]:
%matplotlib inline

import MeCab
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [10]:
def tokenize(text):
    """ MeCab で分かち書きした結果をトークンとして返す """
    wakati = MeCab.Tagger("-O wakati")
    return wakati.parse(text).split(' ')

In [1]:
from pymongo import MongoClient
import keys

mongo = MongoClient(keys.MONGO_URL_MINING)
db = mongo['stock_mining']
collection = db['datahalf']
cur = collection.find({})
orig_data = [row for row in cur]

In [26]:
data = orig_data[:1000]

In [27]:
def get_label(price):
    if price <= 0:
        return -1
    return 1

In [28]:
ir_list = [d['news']['content'] for d in data]
target = [get_label(d['price_data']['percent']) for d in data]
target = np.array(target)

In [30]:
vec = TfidfVectorizer(tokenizer=tokenize, stop_words='english', min_df=1)
tfs = vec.fit_transform(ir_list)
tfs_list = tfs.toarray()
print(tfs_list.shape)

(1000, 1936)


In [35]:
# write into a csv
import csv
with open('ir_vectors.txt', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    for row in tfs_list:
        writer.writerow(row)
with open('ir_vectors_labels.txt', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    for row in target:
        writer.writerow([row])